In [1]:
#Read in the dataset
import pandas as pd
import os
from env import user, password, host

def get_db_url(database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

"""
USAGE: 
Use `from wrangle import wrangle_zillow` at the top of your notebook.
This 
"""
def get_zillow():
    """Seeks to read the cached zillow.csv first """
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        return get_new_zillow_data()

def get_new_zillow_data():
    """Returns a dataframe of all 2017 properties that are Single Family Residential"""

    sql = """
    select 
    bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
    from properties_2017
    join propertylandusetype using (propertylandusetypeid)
    where propertylandusedesc = "Single Family Residential"
    """
    return pd.read_sql(sql, get_db_url("zillow"))


def handle_nulls(df):    
    # We keep 99.41% of the data after dropping nulls
    # round(df.dropna().shape[0] / df.shape[0], 4) returned .9941
    df = df.dropna()
    return df


def optimize_types(df):
    # Convert some columns to integers
    # fips, yearbuilt, and bedrooms can be integers
    df["fips"] = df["fips"].astype(int)
    df["yearbuilt"] = df["yearbuilt"].astype(int)
    df["bedroomcnt"] = df["bedroomcnt"].astype(int)    
    df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
    df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
    return df


def handle_outliers(df):
    """Manually handle outliers that do not represent properties likely for 99% of buyers and zillow visitors"""
    df = df[df.bathroomcnt <= 6]
    
    df = df[df.bedroomcnt <= 6]
    
    df = df[df.calculatedfinishedsquarefeet < 8000]
    
    df = df[df.taxvaluedollarcnt < 1000000]

    return df


def wrangle_zillow():
    """
    Acquires Zillow data
    Handles nulls
    optimizes or fixes data types
    handles outliers w/ manual logic
    returns a clean dataframe
    """
    df = get_zillow_data()

    df = handle_nulls(df)

    df = optimize_types(df)

    df = handle_outliers(df)

    df.to_csv("zillow.csv", index=False)

    return df


In [ ]:
# plotting the baseline residual error (actual - baseline prediction)

# Residuals are the difference between goal and prediction I have

#lm_df["baseline_residuals"] = lm_df["home_value"] - lm_df["baseline"]
#lm_df.head()

In [ ]:
#lm_df["residuals"] = lm_df["home_value"] - lm_df["predictions"]
#lm_df.head()

In [ ]:
# considering creating a function that creates these plots

plt.figure(figsize = (12,6))

plt.subplot(121)
ax = sns.scatterplot(x = "finished_sq_feet", y = "baseline_residuals", data = rmse_validate.sample(1000, random_state = 123))
plt.axhline(y = 0, ls = ':', color = "red")
plt.xlabel('x_variable')
plt.ylabel('Residual')
plt.title('Baseline Residuals')
 # removing axes scientific notation
plt.ticklabel_format(style = "plain")

# making individual plots more readable
ax.figure.set_size_inches(18, 6)

plt.subplot(122)
ax = sns.scatterplot(x = "finished_sq_feet", y = "residuals", data = lm_df.sample(1000, random_state = 123))
plt.axhline(y = 0, ls = ':', color = "red")

plt.xlabel('x_variable')
plt.ylabel('Residual')
plt.title('OLS Model Residuals')
# removing axes scientific notation
plt.ticklabel_format(style = "plain") 

# making individual plots more readable
ax.figure.set_size_inches(18, 8)
plt.tight_layout()

In [ ]:
# calculating the Sum of Squared Errors "SSE" or "Residual Sum of Squares"/RSS for my model
# where the Sum of Squared Errors or Residual Sum of Squares refers to the total "dispersion" of the individual data point error
# this measurement can determine how well or not the created model represents the "actual" target data

lm_df["residual^2"] = lm_df["residuals"].round(2) ** 2
lm_df["baseline_residual^2"] = lm_df["baseline_residuals"].round(2) ** 2
lm_df.head()

In [ ]:
# calculating sum of squared error

SSE_model = sum(lm_df["residual^2"])
print('(Model) SSE = {:.1f}'.format(SSE_model))

In [ ]:
# calculating the explained sum of squares or "ESS"
# explained error
# The ESS (Explained Sum of Squares) is the sum of the difference between each predicted value (y_hat = home_price prediction) and the mean of all actual values (y.mean = home_value mean)

ESS_model = sum((lm_df["predictions"] - lm_df["home_value"].mean()) ** 2)
print('(Model) Explained Sum of Squares = {:.1f}'.format(ESS_model))

In [ ]:
# calculating the total sum of squares errors or "TSS"
# The TSS (Total Sum of Squares) is the sum of difference between the actual final grade and the mean of all final grades
# It can also be derived by summing the ESS and SSE

TSS_model = ESS_model + SSE_model
print('(Model) Total Sum of Squares = {:.1f}'.format(TSS_model))

In [ ]:
# calculating mean squared error or MSE
# we arrive at this by dividing your SSE by the total number of data points
# i.e. the average of your errors that have each been squared

MSE_model = SSE_model/len(lm_df)
print('(Model) Mean Squared Error = {:.1f}'.format(MSE_model))

In [ ]:
# calculating the root mean squared error or RMSE

RMSE_model = sqrt(MSE_model)
print('(Model) Root Mean Squared Error = {:.1f}'.format(RMSE_model))

#### Calculate the sum of squared errors, mean squared error, and root mean squared error for the baseline model (i.e. a model that always predicts the average home_value amount / or "baseline_predictions).

In [ ]:
# calculating sum of squared error

SSE_baseline = sum(lm_df["baseline_residual^2"])
print('(Baseline) SSE = {:.1f}'.format(SSE_baseline)

In [ ]:
# calculating the explained sum of squares or "ESS"
# explained error
# The ESS (Explained Sum of Squares) is the sum of the difference between each predicted value (y_hat = home_price prediction) and the mean of all actual values (y.mean = home_value mean)

ESS_baseline = sum((lm_df["baseline"] - lm_df["home_value"].mean()) ** 2)
print('(Baseline) Explained Sum of Squares = {:.1f}'.format(ESS_baseline))

In [ ]:
# calculating the total sum of squares errors or "TSS"
# The TSS (Total Sum of Squares) is the sum of difference between the actual final grade and the mean of all final grades
# It can also be derived by summing the ESS and SSE

TSS_baseline = ESS_baseline + SSE_baseline
print('(Baseline) Total Sum of Squares = {:.1f}'.format(TSS_baseline))

In [ ]:
# calculating mean squared error or MSE
# we arrive at this by dividing your SSE by the total number of data points
# i.e. the average of your errors that have each been squared

MSE_baseline = SSE_baseline/len(lm_df)
print('(Baseline) Mean Squared Error = {:.1f}'.format(MSE_baseline))

In [ ]:
# calculating the root mean squared error or RMSE

RMSE_baseline = sqrt(MSE_baseline)
print('(Baseline) Root Mean Squared Error = {:.1f}'.format(RMSE_baseline))

In [ ]:
# calculating model variance
# where variance == R^2 or "coefficient of determination" (a measurement from 0 to 1)
# R2: variance in y (target) explained by X (predictor); closer to 1 is better

model_variance = ESS_model/TSS_model
print('Model Variance = {:.1f}'.format(model_variance))

In [ ]:
# calculating the baseline variance

baseline_variance = ESS_baseline/TSS_baseline
print('Baseline Variance = {:.1f}'.format(baseline_variance))

In [ ]:
# let's use sklearn to calculate model variance

model_var = sklearn.metrics.explained_variance_score(lm_df["home_value"], lm_df["predictions"])
model_var.round(3)

In [ ]:
# let's use sklearn to calculate baseline variance

baseline_var = sklearn.metrics.explained_variance_score(lm_df["home_value"], lm_df["baseline"])
baseline_var.round(3) 

In [ ]:
#### This model performs 33% better than baseline

In [ ]:
def compare_sum_of_squares(SSE_baseline, SSE_model):
    if SSE_model >= SSE_baseline:
        print("Model ain't cutting it")
    else:
        print("Model outperforms baseline!")

In [ ]:
#### Write python code that compares the sum of squared errors for your model against the sum of squared errors for the baseline model and outputs whether or not your model performs better than the baseline model.

In [ ]:
#### What is the amount of variance explained in your model?

In [ ]:
### Plot residuals for the linear regression model that you made